In [1]:
print("hello world")

hello world


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()



True

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:

chat_llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", google_api_key=os.getenv("gemini_api_key"))

In [22]:
chat_llm.invoke("What is the capital of France?").content

[{'type': 'text',
  'text': 'The capital of France is **Paris**.',
  'extras': {'signature': 'EqEBCp4BAb4+9vuNS+NLF2goEeTr71jd3dYtU6WNwA1OMG2Y3W8OaTC/rfhy0t0/Q6PEt1qJGFjMewlWnWvYqvVPsGBjpVYMdt40c1FtMM3EQzlYRs9pge5R9eHim4ZJFpHj9rwihVG0Gp07X4v0/rHw9fyA0g81hjCzhYJef9gQi9tNp6liplF1Ne6aYNL/5/y97sJEanYpi8p81p7ffny4xI8='}}]